## Train record files

_train model! (run command on command prompt as it will take lots of time_

>python model_main_tf2.py --model_dir="dataset-sampling\models\my_ssd_resnet50_v1_fpn" --pipeline_config_path="dataset-sampling\models\my_ssd_resnet50_v1_fpn\pipeline.config"

_export model_

>python .\exporter_main_v2.py --input_type image_tensor --pipeline_config_path dataset-sampling\models\my_ssd_resnet50_v1_fpn\pipeline.config --trained_checkpoint_dir dataset-sampling\models\my_ssd_resnet50_v1_fpn --output_directory .\dataset-sampling\exported-models

#### HELPER FUNCTIONS

In [17]:
DATASET_PATH = "dataset-raw"
DATASET_SAMPLING = "dataset-sampling"
NUMBER_IMAGE_TEST = 50
HEIGHT_CV2_IMAGE_RESIZE = 720

In [18]:
import cv2

def cv_imread(file_path):
    # Load image https://stackoverflow.com/a/54491104/4555317
    cv_img = cv2.imdecode(np.fromfile(file_path, dtype=np.uint8), 1)
    return cv_img

def ResizeWithAspectRatio(image, width=None, height=None, inter=cv2.INTER_AREA):
    # stackoverflow copied resize cv2 image
    dim = None
    (h, w) = image.shape[:2]

    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))

    return cv2.resize(image, dim, interpolation=inter)

def quickShow(wt, img):
    resize = ResizeWithAspectRatio(img, height=HEIGHT_CV2_IMAGE_RESIZE)
    cv2.imshow(wt, resize)

## Predict from saved-model

In [9]:
import time, os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
import tensorflow as tf

PATH_TO_SAVED_MODEL = os.path.join(DATASET_SAMPLING,"exported-models","saved_model")

print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
model = tf.saved_model.load(PATH_TO_SAVED_MODEL)
detect_fn = model.signatures['serving_default']

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...Done! Took 8.253122329711914 seconds


In [10]:
category_index = {1: {"id": 1, "name": "bubble"}}
import random
def datasetImage():
    root = DATASET_PATH
    folders = random.sample(os.listdir(root), 1)
    for folder in folders:
        b = os.path.join(root, folder)
        f = random.choice(os.listdir(b))
        ff = os.path.join(b, f)
        print(ff)
        return ff

In [19]:
IMAGE_PATHS = []
for i in range(NUMBER_IMAGE_TEST):
    IMAGE_PATHS.append(datasetImage())

dataset-raw\The Inner Palace Tale Of A Villainess Noble Girl\chapter-6_21.jpg
dataset-raw\I DonΓÇÖT Want A Harem!\chapter-30_42.jpg
dataset-raw\Invasion Of The Immortal Emperor\chapter-25_6.jpg
dataset-raw\Macho Γÿà Max\chapter-8_22.jpg
dataset-raw\Isekai Kenkokuki\chapter-51.2_19.jpg
dataset-raw\The Precious Sister Of The Villainous Grand Duke\chapter-59_1.jpg
dataset-raw\Futari Solo Camp\chapter-26_13.jpg
dataset-raw\My Home Hero\chapter-99_14.jpg
dataset-raw\Trap City\chapter-15_29.jpg
dataset-raw\Argonavis From Bang Dream! Comics\chapter-14_4.jpg
dataset-raw\Spirit KingΓÇÖS Rules\chapter-60_5.jpg
dataset-raw\Solo Spell Caster\chapter-75_7.jpg
dataset-raw\The Elusive Samurai\chapter-45_8.jpg
dataset-raw\Historie\chapter-120_9.jpg
dataset-raw\The Sacrificial Princess\chapter-41_39.jpg
dataset-raw\Tonari No Seki No Hen Na Senpai\chapter-7_9.jpg
dataset-raw\Falling All In You\chapter-100_26.jpg
dataset-raw\Tensei Kizoku No Isekai Boukenroku ~Jichou Wo Shiranai Kamigami No Shito~\chapte

In [21]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
import cv2
#warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))


for image_path in IMAGE_PATHS:

    print('Running inference for {}... '.format(image_path), end='')

    image_np = cv_imread(image_path)

    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.30,
          agnostic_mode=False)
    
    print('Done')
    bgr_img = cv2.cvtColor(image_np_with_detections, cv2.COLOR_RGB2BGR)
    quickShow("im", bgr_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# sphinx_gallery_thumbnail_number = 2

Running inference for dataset-raw\The Inner Palace Tale Of A Villainess Noble Girl\chapter-6_21.jpg... Done
Running inference for dataset-raw\I DonΓÇÖT Want A Harem!\chapter-30_42.jpg... Done
Running inference for dataset-raw\Invasion Of The Immortal Emperor\chapter-25_6.jpg... Done
Running inference for dataset-raw\Macho Γÿà Max\chapter-8_22.jpg... Done
Running inference for dataset-raw\Isekai Kenkokuki\chapter-51.2_19.jpg... Done
Running inference for dataset-raw\The Precious Sister Of The Villainous Grand Duke\chapter-59_1.jpg... Done
Running inference for dataset-raw\Futari Solo Camp\chapter-26_13.jpg... Done
Running inference for dataset-raw\My Home Hero\chapter-99_14.jpg... Done
Running inference for dataset-raw\Trap City\chapter-15_29.jpg... Done
Running inference for dataset-raw\Argonavis From Bang Dream! Comics\chapter-14_4.jpg... Done
Running inference for dataset-raw\Spirit KingΓÇÖS Rules\chapter-60_5.jpg... Done
Running inference for dataset-raw\Solo Spell Caster\chapter-75